# 视频目标检测、视频实例分割

调用 MMTracking 预训练追踪模型，对视频做视频目标检测、视频实例分割。

参考文档：https://github.com/open-mmlab/mmtracking/blob/master/docs/en/quick_run.md

MMtracking 预训练模型库 Model Zoo：https://mmtracking.readthedocs.io/en/latest/model_zoo.html

如果报错`CUDA out of memory.`则重启前面几个代码的`kernel`即可。

作者：同济子豪兄 2022-03-05

## 进入 MMTracking 主目录

In [1]:
import os
os.chdir('mmtracking')
os.listdir()

['.git',
 '.circleci',
 '.dev_scripts',
 '.github',
 '.gitignore',
 '.pre-commit-config.yaml',
 '.readthedocs.yml',
 'CITATION.cff',
 'LICENSE',
 'MANIFEST.in',
 'README.md',
 'README_zh-CN.md',
 'configs',
 'demo',
 'docker',
 'docs',
 'mmtrack',
 'model-index.yml',
 'requirements.txt',
 'requirements',
 'resources',
 'setup.cfg',
 'setup.py',
 'tests',
 'tools',
 'mmtrack.egg-info',
 'checkpoints',
 'outputs',
 'data']

## 视频目标检测 Video Object Detection （VID）

模型的 config 文件需和 checkpoint 权重文件 一一对应，比如下面这样：

| 模型                           | config文件                                                       | checkpoint权重文件                                                   |
| ------------------------------ | ------------------------------------------------------------ | ------------------------------------------------------------ |
| SELSA (ICCV 2019)              | configs/vid/selsa/selsa_faster_rcnn_r101_dc5_1x_imagenetvid.py | https://download.openmmlab.com/mmtracking/vid/selsa/selsa_faster_rcnn_r101_dc5_1x_imagenetvid/selsa_faster_rcnn_r101_dc5_1x_imagenetvid_20201218_172724-aa961bcc.pth |
| Temporal RoI Align (AAAI 2021) | configs/vid/temporal_roi_align/selsa_troialign_faster_rcnn_x101_dc5_7e_imagenetvid.py | https://download.openmmlab.com/mmtracking/vid/temporal_roi_align/selsa_troialign_faster_rcnn_x101_dc5_7e_imagenetvid/selsa_troialign_faster_rcnn_x101_dc5_7e_imagenetvid_20210822_164036-4471ac42.pth |

### 命令行

In [3]:
!python ./demo/demo_vid.py \
    ./configs/vid/selsa/selsa_faster_rcnn_r101_dc5_1x_imagenetvid.py \
    --checkpoint https://download.openmmlab.com/mmtracking/vid/selsa/selsa_faster_rcnn_r101_dc5_1x_imagenetvid/selsa_faster_rcnn_r101_dc5_1x_imagenetvid_20201218_172724-aa961bcc.pth \
    --input data/mot_people_short.mp4 \
    --output outputs/B1_VID_SELSA.mp4 \
    --score-thr 0.4 \
    --thickness 2

2022-04-21 13:00:29,908 - mmtrack - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet101'}
2022-04-21 13:00:29,909 - mmcv - INFO - load model from: torchvision://resnet101
2022-04-21 13:00:29,909 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet101
2022-04-21 13:00:30,094 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2022-04-21 13:00:30,121 - mmtrack - INFO - initialize ChannelMapper with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-04-21 13:00:30,182 - mmtrack - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-04-21 13:00:30,198 - mmtrack - INFO - initialize SelsaBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 

### Python API

In [4]:
import mmcv
import tempfile
from mmtrack.apis import inference_vid, init_model

# 输入输出视频路径
input_video = 'data/mot_people_short.mp4'
output = 'outputs/B2_VID_SELSA.mp4'

# 指定 config 配置文件 和 模型权重文件，创建模型
vid_config = './configs/vid/selsa/selsa_faster_rcnn_r101_dc5_1x_imagenetvid.py'
vid_checkpoint = 'https://download.openmmlab.com/mmtracking/vid/selsa/selsa_faster_rcnn_r101_dc5_1x_imagenetvid/selsa_faster_rcnn_r101_dc5_1x_imagenetvid_20201218_172724-aa961bcc.pth'
vid_model = init_model(vid_config, vid_checkpoint, device='cuda:0')

# 读入待预测视频
imgs = mmcv.VideoReader(input_video)
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name

# 逐帧输入模型预测
for i, img in enumerate(imgs):
    result = inference_vid(vid_model, img, frame_id=i)
    vid_model.show_result(
            img,
            result,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')
    prog_bar.update()

# 生成预测视频
print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2022-04-21 13:01:13,551 - mmtrack - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet101'}
2022-04-21 13:01:13,554 - mmcv - INFO - load model from: torchvision://resnet101
2022-04-21 13:01:13,555 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet101
2022-04-21 13:01:13,791 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2022-04-21 13:01:13,823 - mmtrack - INFO - initialize ChannelMapper with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-04-21 13:01:13,882 - mmtrack - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-04-21 13:01:13,901 - mmtrack - INFO - initialize SelsaBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 

load checkpoint from http path: https://download.openmmlab.com/mmtracking/vid/selsa/selsa_faster_rcnn_r101_dc5_1x_imagenetvid/selsa_faster_rcnn_r101_dc5_1x_imagenetvid_20201218_172724-aa961bcc.pth
[                                                  ] 0/99, elapsed: 0s, ETA:

/home/featurize/work/MMTracking教程/0420/mmtracking/mmtrack/datasets/pipelines/formatting.py:138: UserWarning: The 'ConcatVideoReferences' class will be deprecated in the future, please use 'ConcatSameTypeFrames' instead
  "The 'ConcatVideoReferences' class will be deprecated in the "


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 99/99, 4.6 task/s, elapsed: 22s, ETA:     0s
 making the output video at outputs/B2_VID_SELSA.mp4 with a FPS of 24.652929091701427
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 99/99, 25.9 task/s, elapsed: 4s, ETA:     0s


## 视频实例分割 Video Instance Segmentat （VIS）

### 命令行

In [5]:
!python demo/demo_mot_vis.py \
        configs/vis/masktrack_rcnn/masktrack_rcnn_r50_fpn_12e_youtubevis2019.py \
        --checkpoint https://download.openmmlab.com/mmtracking/vis/masktrack_rcnn/masktrack_rcnn_r50_fpn_12e_youtubevis2019/masktrack_rcnn_r50_fpn_12e_youtubevis2019_20211022_194830-6ca6b91e.pth \
        --input data/mot_people_short.mp4 \
        --output outputs/B3_VIS_masktrack_rcnn.mp4

2022-04-21 13:01:54,504 - mmtrack - INFO - initialize MaskRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth'}
2022-04-21 13:01:54,505 - mmcv - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth
2022-04-21 13:01:54,505 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth
Downloading: "https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth" to /home/featurize/.cache/torch/hub/checkpoints/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth
100%|████████████████████████████████████████| 170M/170M [00:01<00:00, 92.8MB/s]
2022-04-21 13:01:56,586 - mmcv - W

### Python API

In [6]:
import mmcv
import tempfile
from mmtrack.apis import inference_mot, init_model

# 输入输出视频路径
input_video = 'data/mot_people_short.mp4'
output = 'outputs/B4_VIS_masktrack_rcnn.mp4'

# 指定 config 配置文件 和 模型权重文件，创建模型
vis_config = './configs/vis/masktrack_rcnn/masktrack_rcnn_r50_fpn_12e_youtubevis2019.py'
vis_checkpoint = 'https://download.openmmlab.com/mmtracking/vis/masktrack_rcnn/masktrack_rcnn_r50_fpn_12e_youtubevis2019/masktrack_rcnn_r50_fpn_12e_youtubevis2019_20211022_194830-6ca6b91e.pth'
vis_model = init_model(vis_config, vis_checkpoint, device='cuda:0')

# 读入待预测视频
imgs = mmcv.VideoReader(input_video)
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name

# 逐帧输入模型预测
for i, img in enumerate(imgs):
    result = inference_mot(vis_model, img, frame_id=i)
    vis_model.show_result(
            img,
            result,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')
    prog_bar.update()
    

print(f'\n making the output video at {output} with a FPS of {imgs.fps}')

# 生成预测视频
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2022-04-21 13:02:31,929 - mmtrack - INFO - initialize MaskRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth'}
2022-04-21 13:02:31,930 - mmcv - INFO - load model from: https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth
2022-04-21 13:02:31,931 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_1x_coco/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth
2022-04-21 13:02:32,061 - mmcv - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([41, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size(

load checkpoint from http path: https://download.openmmlab.com/mmtracking/vis/masktrack_rcnn/masktrack_rcnn_r50_fpn_12e_youtubevis2019/masktrack_rcnn_r50_fpn_12e_youtubevis2019_20211022_194830-6ca6b91e.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 99/99, 4.2 task/s, elapsed: 23s, ETA:     0s
 making the output video at outputs/B4_VIS_masktrack_rcnn.mp4 with a FPS of 24.652929091701427
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 99/99, 25.5 task/s, elapsed: 4s, ETA:     0s
